This notebook contains an implementation of the block SVD algorithm described in [LAWN 283](http://www.netlib.org/lapack/lawnspdf/lawn283.pdf)

## Block and Unblock Arrays

In [1]:
################################################################
###### Imports
################################################################

# For Linear Algebra
#import Base.LinAlg: QRPackedQ, Ac_mul_B, Ac_mul_Bc!, ctranspose
# import Base.*

# using LinearAlgebra

In [2]:
# # For Visualization 
# using PyCall
# import PyCall: PyObject
# import PyPlot
# @pyimport matplotlib as matplotlib
# @pyimport matplotlib.patches as patches
# @pyimport matplotlib.animation as anim

# #const plt = PyPlot # To prevent function name conflicts (spy)

# using PyPlot

# # Fix path to FFmpeg
# matplotlib.rcParams["animation.ffmpeg_path"] = "/usr/local/bin/ffmpeg"

In [3]:
using LinearAlgebra

using Plots

myspy(A) = (display(heatmap(A, yflip=true)); sleep(0.5))

myspy (generic function with 1 method)

In [4]:
################################################################
###### Function Overrides
################################################################

# JiaHao's Fix
#Base.adjoint(Q::Base.LinAlg.QRCompactWYQ) = full(Q)'

function block(A,s1=5,s2=5)  
    m, n=size(A)
    mi=[i:min(i+s1-1,m) for i=1:s1:m]
    ni=[j:min(j+s2-1,n) for j=1:s2:n]
    [A[i,j] for i in mi, j in ni]
end

#unblock(A)=hvcat(tuple([size(A,2) for i=1:size(A,1)]...), A'...) 

unblock(A) = vcat([hcat(A[i,:]...) for i in 1:size(A, 1)]...)

    


# Key BLAS3 block operator
# A few block apply Q's
# Ac_mul_B(Q::QRPackedQ,B)=hcat([block(Q'*unblock(B[:,j])) for j in 1:size(B,2)]...)           
# *{T<:Matrix}(A::Array{T},Q::QRPackedQ)=vcat([block(unblock(A[i,:])*Q) for i in 1:size(A,1)]...)   
                                        
# # A few block qrQ's
qrQ(A::Matrix) = qr(A).Q
qrQ(A) = qrQ(unblock(A))  
qrQc(A) = qrQ(unblock(A)')
                                 
# spyt{T<:Number}(A::Array{T})= plt.spy(A, precision=sqrt(eps(T)), alpha=0.8)  
# spy{T<:Matrix}(A::Array{T})= plt.spy(unblock(A), precision=sqrt(eps()), alpha=0.7)  # Threshold for convenience


################################################################
###### Convenience Drawing Functions
################################################################

qrQc (generic function with 1 method)

In [5]:
# plt.autoscale(enable=true)
# plt.gca()[:set_xlim](0, 25)
# plt.gca()[:set_ylim](25, 0)

# offset = 1.4

# # rect_color(horizontal_offset, vertical_offset, width, height)
# rect_red(x0, y0, dx, dy)=  plt.gca()[:add_patch](patches.Rectangle((x0-offset,y0-offset),dx,dy,linewidth=4,facecolor="none",edgecolor= "red"))
# rect_blue(x0, y0, dx, dy)= plt.gca()[:add_patch](patches.Rectangle((x0-offset,y0-offset),dx,dy,linewidth=6,facecolor="none",edgecolor= "SlateBlue"))
# rect_gray(x0, y0, dx, dy)= plt.gca()[:add_patch](patches.Rectangle((x0-offset,y0-offset),dx,dy,linewidth=2,facecolor="none",edgecolor="#aaaaaa"))

# rect_red(yr, xr)=rect_red(start(xr),start(yr),length(xr),length(yr))
# rect_blue(yr, xr)=rect_blue(start(xr),start(yr),length(xr),length(yr))
# rect_gray(yr, xr)=rect_gray(start(xr),start(yr),length(xr),length(yr))

In [6]:
function BandBidiagonal(A::Array{Array{Float64,2},2}, record_video=false, video_file="band_bidiagonal.mp4") 
    
    # Video Settings
#     if record_video
#         video = anim.FFMpegWriter(fps=1, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
#         video[:setup](plt.gcf(), video_file, dpi=100)
#     end
    
    results = [ ]
    
    (m, n) = size(A)
    s = size(A[1,1], 1) #Square block size

    M,N = size(unblock(A))
    
    for i=1:ceil(Int,N/s)

        Q=qrQ(A[i,i])
        
        
        A[i,:] = block(Q' * unblock(A[i:i,:])) # Q' acts like a square matrix even though Q was "thin" 
        
        push!(results, unblock(A))
        
        for j=(i+1):ceil(Int,N/s)
            Q = qrQ(A[[i,j],i])         # QR for a "domino"=[◥;◼]
            A[[i,j],:] = block(Q' * unblock(A[[i,j],:])) # Q' acts like a square matrix even though Q was "thin" 

#           rect_blue(((i-1) * s)+1,(i-1)*s+1, s, s)
#           rect_red(((i-1) * s)+1,(j-1)*s+1, s, s)
#           record_video && video[:grab_frame]()
            
            push!(results, unblock(A))

            
        end
    
        if i<n    
            Q = qrQ(A[i,i+1]')            # We really need an LQ
            A[:,i+1] = block(unblock(A[:,(i+1):(i+1)]) * Q) # A[:,i+1]*=Q
        end    
        
        for j=(i+2):ceil(Int,N/s)
          Q = qrQc(A[i:i,[i+1,j]])      # LQ for a "domino"=[◥;◼]'                       
          A[:,[i+1,j]] = block(unblock(A[:,[i+1,j]]) * Q) #  A[:, i+1] = A[:, i+1]'
#           plt.clf()
#           spy(A)
#           rect_blue(((i) * s)+1,(i-1)*s+1, s, s)
#           rect_red((j-1)*s + 1, (i-1)*s + 1, s, s)
#           record_video && video[:grab_frame]()
#           record_video && video[:grab_frame]()
            
            push!(results, unblock(A))

            
        end
        
    end

#     if record_video
#         plt.clf()
#         spy(A)
#         rect_blue((n-1)*s + 1, (n-1)*s + 1, s, s)
#         video[:grab_frame]()
#         video[:grab_frame]()
#         video[:finish]()
#     end
#     plt.clf()
    
    return A, results
    
end

BandBidiagonal (generic function with 3 methods)

In [7]:
A = block(randn(20, 20))
A_after, results = BandBidiagonal(A, false)
# spy(A_after)
# [svdvals(unblock(A))[1:5] svdvals(unblock(A_after))[1:5]]

(Array{Float64,2}[[-5.38716 -0.0929759 … -1.59759 0.828291; 1.11022e-16 -4.34371 … -1.27403 0.36976; … ; 0.0 -5.55112e-17 … -3.8163 -1.75303; -1.46069e-33 0.0 … 2.77556e-17 4.03192] [3.09901 6.16298e-33 … -3.46945e-18 -6.93889e-18; 1.51293 -4.01627 … 1.11022e-16 5.55112e-17; … ; 0.463722 -2.39552 … 3.0039 4.93038e-32; -0.255471 1.98672 … -0.579302 2.29524] [1.58103e-17 -4.95545e-17 … -5.18307e-17 3.1467e-17; 8.89734e-17 -5.13286e-17 … -2.70773e-17 2.88419e-17; … ; -1.06449e-16 -1.12574e-16 … -2.80598e-17 -2.76757e-16; 5.60574e-17 1.42455e-17 … -8.50382e-17 2.42299e-17] [1.93841e-16 -4.57496e-17 … -5.10755e-17 -7.81115e-17; -1.78657e-17 1.4685e-16 … 6.85831e-17 3.90557e-17; … ; 3.05287e-17 9.75395e-17 … -7.86273e-17 -1.30439e-17; 1.35476e-16 -1.32234e-16 … -4.05729e-17 -1.96663e-16]; [-1.8051e-16 1.79802e-16 … 9.71881e-17 -1.123e-16; -8.96465e-17 2.84662e-17 … -6.13305e-17 -2.65324e-16; … ; -5.02127e-18 -1.37765e-16 … -1.44673e-16 4.7089e-16; 1.19e-16 -1.89469e-16 … -5.36329e-16 -2.9299

In [8]:
using Interact

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b0fdfd1e8b92c5dd0fe995878315ede48eb64cc7-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b0fdfd1e8b92c5dd0fe995878315ede48eb64cc7-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [9]:
@manipulate for i in slider(1:length(results), value=1)
    A = results[i]
    heatmap(abs.(A) .< 1e-9, yflip=true, aspect_ratio=1)
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
1), Observable{Int64} with 2 listeners. Value:
1, Scope("knockout-component-0d8d51df-4b1c-4e0e-9e87-fdf8b0ecfff5", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>13,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/alanedelman/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/alanedelman/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/alanedelman/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/alanedelman/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/alanedelman/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":1}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-0d8d51df-4b1c-4e0e-9e87-fdf8b0ecfff5\",\"id\":\"ob_02\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-0d8d51df-4b1c-4e0e-9e87-fdf8b0ecfff5\",\"id\":\"ob_01\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"va

In [10]:
svdvals(unblock(A)) ≈ svdvals(unblock(A_after))

true

In [11]:
A = block(randn(15, 15), 5, 5);
A = BandBidiagonal(A);
#println("hello")

s = size(A[1, 1], 1) # square block size
n = size(A, 1)      # number of blocks
A = unblock(A)

round.(A, digits=3)

MethodError: MethodError: no method matching getindex(::Tuple{Array{Array{Float64,2},2},Array{Any,1}}, ::Int64, ::Int64)
Closest candidates are:
  getindex(::Tuple, ::Int64) at tuple.jl:24
  getindex(::Tuple, ::Real) at tuple.jl:25
  getindex(::Tuple, !Matched::AbstractUnitRange{#s57} where #s57<:Real) at range.jl:268
  ...

In [12]:
i = 1
#display(A[i,i+(1:s)])
Q = qrQ((A[i, i .+ (1:s)])') #odd
#println("hello3")
A[i.+(0:s), i.+(1:s)] *= Q

#println("hello4")
#rect_red(i:i,i+(1:s))  #QR from this block
#rect_blue(i+(0:s),i+(1:s)) #QR applied to this block

for i=1:s:((n-2)*s)
    #println("hello2")
    Q = qrQ(A[i.+(1:s), i+1:i+1]) #even
    A[i.+(1:s), i.+(1:2s)] = Q * A[i.+(1:s), i.+(1:2s)]
    
#     rect_red(i+(1:s),i+1:i+1)  #QR from this block
#     rect_blue(i+(1:s),i+(1:2s)) #QR applied to this block
    
    Q = qrQ(A[i+1, (i+s).+(1:s)]')  #odd
    A[i.+(1:2s), (i+s).+(1:s)] *= Q
#     rect_red(i+1:i+1,i+s+(1:s))  #QR from this block
#     rect_blue(i+(1:2s),i+s+(1:s)) #QR applied to this block
end

i = (n-2)*s+1
Q = qrQ(A[i.+(1:s), i+1:i+1]) #even
A[i.+(1:s), i+1:end] = Q * A[i.+(1:s), i+1:end]
# rect_red(i+(1:s),i+1:i+1)  #QR from this block
# rect_blue(i+(1:s),i+1:(n*s)) #QR applied to this block
Q = qrQ(A[i+1, i+s+1:end]')  #odd
A[i+1:end, i+s+1:end] *= Q
# rect_red(i+1:i+1,i+s+1:(n*s))  #QR from this block
# rect_blue(i+1:(n*s),i+s+1:(n*s)) #QR applied to this block
i = (n-1)*s+1
Q = qrQ(A[i+1:end, i+1:i+1]) #even
A[i+1:end,i+1:end] = Q * A[i+1:end,i+1:end]
# rect_red(i+1:(n*s),i+1:i+1)  #QR from this block
# rect_blue(i+1:(n*s),i+1:(n*s)) #QR applied to this block
# spyt(A)

UndefVarError: UndefVarError: s not defined

In [13]:
function block_bidiagonalize(M, s1=5, s2=5; record_video=false, video_file="blocksvd.mp4")
#     if record_video 
#         video = anim.FFMpegWriter(fps=6, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
#         video[:setup](plt.gcf(), video_file, dpi=100)
#     end
    A, results = BandBidiagonal(block(M, s1, s2), record_video)
    
    s = size(A[1,1],1) #Square block size
    n = size(A,1)      #Number of blocks
    A = unblock(A)
    
    # results = [ ]

    for j=1:n*s-2    #bulge chasing: elimination on row j+1
        ebb = min(j+s, n*s)
        
        #@show A[j, j+1:ebb]
        
        Q = qrQ(A[j:j, j+1:ebb]')           #xGBCW1
        
        #@show size(A[j:ebb, j+1:ebb]), size(Q), typeof(Q), Q
        
        A[j:ebb, j+1:ebb] *= Q
        
        push!(results, copy(A))

#         spyt(A)
#         rect_red(j:j,j+1:ebb)  #QR from this block
#         rect_blue(j:ebb,j+1:ebb) #QR applied to this block
#         record_video && video[:grab_frame]()

        lastb = ((n-floor(div(j,s)) - 1) * s) + j #index of last block
        for i=j:s:lastb
            ebb, ebp1=min(i+s, n*s), min(i+2s, n*s) #index of end of block and its neighbor
            
            Q=qrQ(A[i+1:ebb, i+1:i+1])  #xGBCW2
            A[i+1:ebb, i+1:ebp1] = Q * A[i+1:ebb, i+1:ebp1]
            
            push!(results, copy(A))
            
#             spyt(A)
#             rect_red(i+1:ebb,i+1:i+1)  #QR from this block
#             rect_blue(i+1:ebb,i+1:ebp1) #QR applied to this block
#             record_video && video[:grab_frame]()
            i==lastb && break
            Q=qrQ(A[i+1:i+1, i+s+1:ebp1]') #xGBCW3
            A[i+1:ebp1, i+s+1:ebp1] *= Q
            
            push!(results, copy(A))
#             spyt(A)
#             rect_red(i+1:i+1,i+s+1:ebp1)  #QR from this block
#             rect_blue(i+1:ebp1,i+s+1:ebp1) #QR applied to this block
#             record_video && video[:grab_frame]()
        end
    
        #Gray out stuff
#         plt.clf()
#         endb = min(j+s,n*s)
# #         rect_gray(j:j,j+1:endb)
# #         rect_gray(j:endb,j+1:endb)
#         for i=j:s:lastb
#             endb, endbp1 = min(i+s,n*s), min(i+2s,n*s)
# #             rect_gray(i+1:endb,i+1:i+1)
# #             rect_gray(i+1:endb,i+1:endbp1)
# #             rect_gray(i+1:i+1,i+s+1:endbp1)
# #             rect_gray(i+1:endbp1,i+s+1:endbp1)
#         end
    end
    
#     plt.clf()
#     spyt(A)
#     if record_video
#         video[:grab_frame]()
#         video[:finish]()
#     end
    
    return A, results
end

block_bidiagonalize (generic function with 3 methods)

In [14]:
M = randn(15, 15)
A, results = block_bidiagonalize(M, 5, 5, record_video=false)
#spyt(A)

#round.(A, digits=3)

([-2.69886 3.47094 … 3.46173e-16 -2.73677e-16; 2.08167e-17 -4.42642 … -9.9209e-17 -8.72381e-17; … ; 1.49595e-16 1.36305e-16 … -1.32478 0.85179; 1.20364e-16 -1.05301e-16 … -2.22045e-16 -0.886986], Any[[-1.51894 -1.42887 … 1.48953 -0.847355; 4.44089e-16 -1.99117 … 1.15452 2.06057; … ; 0.731133 -1.00083 … 1.14768 -0.543352; 0.512369 1.94809 … -1.05638 -1.14444], [2.46779 1.43576 … -0.723403 1.15271; 5.55112e-17 2.83985 … -1.28663 -1.16922; … ; 0.731133 -1.00083 … 1.14768 -0.543352; 0.512369 1.94809 … -1.05638 -1.14444], [-2.69886 -1.47572 … 0.52277 -0.505041; 2.08167e-17 -3.85479 … 2.30912 1.65778; … ; 0.0 0.0 … 2.08304 -2.37627; 0.0 0.0 … -0.0752268 -0.782457], [-2.69886 -1.47572 … -1.11022e-16 0.0; 2.08167e-17 -3.85479 … 0.0 2.22045e-16; … ; 0.0 0.0 … 2.07153 -2.47175; 0.0 0.0 … 1.3195 0.109977], [-2.69886 -1.47572 … -1.11022e-16 0.0; 2.08167e-17 -3.85479 … 0.0 2.22045e-16; … ; 0.0 0.0 … 2.07153 -2.47175; 0.0 0.0 … 1.3195 0.109977], [-2.69886 -1.47572 … -1.11022e-16 0.0; 2.08167e-17 -3.

In [15]:
@manipulate for i in slider(1:length(results), value=1)
    A = results[i]
    heatmap(abs.(A) .< 1e-9, yflip=true, aspect_ratio=1, c=:grays)  # or :blues
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:i=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
1), Observable{Int64} with 2 listeners. Value:
1, Scope("knockout-component-b5f8ce05-a2d0-42ea-a917-ecfaaa7a95f1", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>59,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/alanedelman/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/Users/alanedelman/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/Users/alanedelman/.julia/packages/InteractBase/Q4IkI/src/../assets/all.js", "/Users/alanedelman/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css", "/Users/alanedelman/.julia/packages/InteractBulma/Ohu5Y/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":1}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-b5f8ce05-a2d0-42ea-a917-ecfaaa7a95f1\",\"id\":\"ob_07\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-b5f8ce05-a2d0-42ea-a917-ecfaaa7a95f1\",\"id\":\"ob_06\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"va

In [16]:
svdvals(A) ≈ svdvals(M)

true

In [17]:
function block_svdvals(M,s1=5,s2=5)
    A=block_bidiagonalize(M,s1,s2)
    B=Bidiagonal(diag(A), diag(A,1), true)
    svdvals(B)
end

block_svdvals(M) 
svdvals(M)

MethodError: MethodError: no method matching diag(::Tuple{Array{Float64,2},Array{Any,1}})
Closest candidates are:
  diag(!Matched::BitArray{2}) at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/bitarray.jl:80
  diag(!Matched::SymTridiagonal) at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/tridiag.jl:145
  diag(!Matched::SymTridiagonal, !Matched::Integer) at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/tridiag.jl:145
  ...

In [18]:
# ;ls -l *.mp4
embed_video(filename)=display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
                            base64encode(open(readbytes,filename)),"""" type="video/mp4"></video>"""))
embed_video("blocksvd.mp4")
embed_video("band_bidiagonal.mp4")

UndefVarError: UndefVarError: readbytes not defined

In [19]:
using DistributedArrays

In [20]:
A = rand(4,4)

4×4 Array{Float64,2}:
 0.91298   0.224599  0.600675    0.267523 
 0.898929  0.677414  0.673169    0.0507825
 0.470681  0.948454  0.7127      0.59086  
 0.333378  0.258253  0.00786634  0.0861263

In [ ]:
A[1,2][1,1]

In [ ]:
# A[1,2] # is this a block or an element?